# 2. Data Preparation

2장에 대한 수정 작업을 develop 브랜치에서 진행하고 있음. 계속 진행하는 중. 한번 더 수정

## 2.1 Travel Demand Data (Trip record data)

### 2.1.1 활용 데이터

- 서울시에서 제공하는 수도권 생활이동 데이터를 활용하였다. 
- https://data.seoul.go.kr/dataVisual/seoul/capitalRegionLivingMigration.do
- 컬럼의 명세는 아래와 같다. 행정동 단위의 O-D 통행량 및 통행시간 정보를 제공해주고 있다. 더 상세하게는 내/외국인 구분, 국적, 이동목적과 같은 정보도 제공을 해준다.


| 순번 | 영문 컬럼명 | 컬럼 설명 | NULL 여부 | NULL 대체값 | 형식 | 규칙 | 데이터 허용범위 | 비고 |
|------|------------|-----------|-----------|-------------|------|------|-----------------|------|
| 1 | O_ADMDONG_CD | 출발 행정동 | X | - | STRING | - | - | 행안부 8자리 코드체계 |
| 2 | D_ADMDONG_CD | 도착 행정동 | X | - | STRING | - | - | 행안부 8자리 코드체계 |
| 3 | ST_TIME_CD | 출발 시간 | X | - | STRING | - | - | 7~9시/17시~19시는 20분단위, 그 외 1시간 단위 |
| 4 | FNS_TIME_CD | 도착 시간 | X | - | STRING | - | - | 7~9시/17시~19시는 20분단위, 그 외 1시간 단위 |
| 5 | IN_FORN_DIV_NM | 내/외국인 구분 | X | - | STRING | - | - | 내국인, 단기외국인, 장기외국인 |
| 6 | FORN_CITIZ_NM | 국적 | X | - | STRING | - | - | 　 |
| 7 | MOVE_PURPOSE | 이동 목적 | X | - | STRING | - | - | 1: 출근, 2 : 등교, 3: 귀가, 4: 쇼핑, 5: 관광, 6: 병원, 7: 기타 |
| 8 | MOVE_DIST | 평균 이동 거리(m) | X | - | DOUBLE | - | - | 　 |
| 9 | MOVE_TIME | 평균 이동 시간(분) | X | - | DOUBLE | - | - | 　 |
| 10 | CNT | 이동인구 수 | X | - | DOUBLE | (18,2) | - | 　 |
| 11 | ETL_YMD | 기준 년월 일 | X | - | STRING | yyyyMMdd | 데이터 기준 당일 | - |

데이터셋 명: 출도착 행정동별 이동 목적 데이터  ·
DB 명: PURPOSE_ADMDONG3

### 2.1.2 데이터 읽기 및 간단한 전처리

Travel Demand에 대한 상세한 데이터의 처리는 향후, 4장에서 보다 상세하게 다룰 예정이다. 

## 2.2 Travel Time Data

## 2.3 Road Network and Geometry

### 2.3.1 Open Street Map

[오픈스트리트맵 한국](https://osm.kr/usage/)에서는 대한민국의 OSM을 이용하는 상세한 방법에 대한 메뉴얼을 제공해주고 있다. 상세한 내용은 이곳을 참고하면 되며, 본 장에서는 한국의 OSM 데이터를 불러와서 `OSMnx` 패키지로 간략히 분석하고 시각화 하는 작업을 진행한다.  

OSM을 이용해서 수도권 지역의 도로 데이터를 가져오고, 시각화 하는 작업을 수행  
OSMnx 이용해서 그래프 분석하고, 지난번 세미나 때 했던 간소화 하는 작업도 해볼 수 있지 않을까



## 2.4 GTFS (General Transit Feed Specification)

- 대중교통 네트워크 분석을 위한 데이터
- 향후 업데이트 예정